<a href="https://colab.research.google.com/github/emasri/tcol/blob/master/td6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Chcek notebook's internet speed

In [ ]:
!sudo apt install speedtest-cli
!speedtest-cli

###Time out preventer

Run this code to prevent Notebook Timeout

In [ ]:
#from os import makedirs
#makedirs("/root/.config/rclone", exist_ok = True)
import IPython
from google.colab import output

display(IPython.display.Javascript('''
function ClickConnect(){
  btn = document.querySelector("colab-connect-button")
  if (btn != null){
    console.log("Click colab-connect-button"); 
    btn.click() 
    }
    
    btn = document.getElementById('ok')
    if (btn != null){
    console.log("Click reconnect"); 
    btn.click() 
    }
  }
  
setInterval(ClickConnect,60000)
'''))

### Start session

In [2]:

!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent

!apt install python3-libtorrent

import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)

sett = {'allow_multiple_connections_per_ip': True,
  'active_downloads': -1}

ses.apply_settings(sett)

downloads = []

###Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

###Add torrent file

In [ ]:
from google.colab import files

source = files.upload()
params = {
    "save_path": "/content/drive/My Drive/Library/Downloads",
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

###Add multi torrent magnet

In [ ]:
import time

params = {"save_path": "/content/drive/My Drive/Library/Downloads"}
magnet_link = ""
handle=""

while True:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    handle = lt.add_magnet_uri(ses, magnet_link, params)
    handle.stop_when_ready(True)
    handle.set_sequential_download(True)
    downloads.append(
        handle
    )

###Download metadata multi torrent


In [ ]:
checks=[]
for h in downloads:
  h.resume()
  checks.append(h)

while (checks):
  for h in checks:
    if h.has_metadata():
      checks.remove(h)

ses.pause()

###List all files multi 

In [ ]:
print("All files:")

for h in downloads:
  print("")
  print("  "+h.name())
  if(h.has_metadata()):
    torinfo = h.get_torrent_info()
    torfiles = torinfo.files()
    for x in range(torfiles.num_files()):
      print("   {:3d}".format(x),torfiles.file_name(x))
  else:
    print("    Metadata not recieved yet.")


###Filter multi file

In [ ]:
skip_list = ['RARBG.txt',
             'RARBG_DO_NOT_MIRROR.exe',
             'readme.txt'
             ]

for h in downloads:
  if(h.has_metadata()):
    torinfo = h.get_torrent_info()
    files = torinfo.files()
    for x in range(files.num_files()):
      if any(i in files.file_name(x) for i in skip_list):
        h.file_priority(x,0)


###Check download files multi 

In [ ]:
print("Download files:")

for h in downloads:
  print("")
  print("  "+h.name())
  if(h.has_metadata()):
    priorities = h.file_priorities()
    torinfo = h.get_torrent_info()
    files = torinfo.files()
    for x in range(files.num_files()):
      if (priorities[x]>0):
        print("   Download", files.file_name(x))
  else:
    print("    Metadata not recieved yet.")


###Add single torrent magnet

In [4]:
import time

ses.resume()

params = {"save_path": "/content/drive/My Drive/Library/Downloads"}
magnet_link = ""

magnet_link = input("Enter Magnet Link Or Type Exit: ")

handle = lt.add_magnet_uri(ses, magnet_link, params)
downloads.append(
    handle
)

#handle.stop_when_ready(True)
handle.set_sequential_download(True)

while (not handle.has_metadata()):
  time.sleep(0.2)

ses.pause () 

###File list single

In [ ]:
torinfo = handle.get_torrent_info()
files = torinfo.files()

for x in range(files.num_files()):
  print(x,files.file_name(x))

###Choose files with index

In [6]:
start=82 #@param {type:"integer"}
end=102 #@param {type:"integer"}

priorities = handle.file_priorities()

for index in range(0,len(priorities)):
  if index >= start and index <= end:
    handle.file_priority(index,4)
  else:
    handle.file_priority(index,0)

range2Start=285 #@param {type:"integer"}
range2End=306 #@param {type:"integer"}

if(range2End-range2Start > 0):
  for index in range(0,len(priorities)):
    if index >= range2Start and index <= range2End:
      handle.file_priority(index,4)



###Check choosen files

In [ ]:
priorities = handle.file_priorities()
torinfo = handle.get_torrent_info()
files = torinfo.files()

print("Files to download:")

for x in range(files.num_files()):
  if (priorities[x]>0):
    print("Download",x,files.file_name(x))

In [ ]:
status = handle.status()
print(status.sequential_download)

###Recheck

In [ ]:
handle.force_recheck()
#handle.resume()

###Add trackers

In [ ]:
new_trackers = ["udp://tracker.opentrackr.org:1337/announce"
,"udp://tracker.internetwarriors.net:1337/announce"
,"udp://exodus.desync.com:6969/announce"
,"udp://tracker.cyberia.is:6969/announce"
,"udp://3rt.tace.ru:60889/announce"
,"http://5rt.tace.ru:60889/announce"
,"udp://explodie.org:6969/announce"
,"udp://47.ip-51-68-199.eu:6969/announce"
,"udp://opentracker.i2p.rocks:6969/announce"
,"http://open.acgnxtracker.com:80/announce"
,"udp://www.torrent.eu.org:451/announce"
,"udp://tracker.torrent.eu.org:451/announce"
,"udp://tracker.tiny-vps.com:6969/announce"
,"udp://tracker.ds.is:6969/announce"
,"udp://retracker.lanta-net.ru:2710/announce"
,"udp://open.stealth.si:80/announce"
,"udp://tracker.moeking.me:6969/announce"
,"udp://ipv4.tracker.harry.lu:80/announce"
,"http://tracker.dler.org:6969/announce"
,"http://rt.tace.ru:80/announce"
,"udp://93.158.213.92:1337/announce"
,"udp://138.255.103.83:1337/announce"
,"udp://208.83.20.20:6969/announce"
,"udp://193.218.118.220:6969/announce"
,"udp://79.111.118.59:60889/announce"
,"http://93.157.234.32:60889/announce"
,"udp://184.105.151.164:6969/announce"
,"udp://51.68.199.47:6969/announce"
,"udp://51.81.46.170:6969/announce"
,"udp://91.216.110.52:451/announce"
,"udp://89.234.156.205:451/announce"
,"udp://5.206.60.196:6969/announce"
,"udp://5.226.148.20:6969/announce"
,"udp://37.235.174.46:2710/announce"
,"udp://185.181.60.67:80/announce"
,"udp://138.201.150.56:6969/announce"
,"udp://51.15.40.114:80/announce"
,"http://168.119.237.9:6969/announce"
,"http://195.201.31.194:80/announce"
,"https://trakx.herokuapp.com:443/announce"
,"https://w.wwwww.wtf:443/announce"
,"https://tracker.tamersunion.org:443/announce"
,"https://tracker.sloppyta.co:443/announce"
,"https://tracker.renfei.net:443/announce"
,"https://tracker.nitrix.me:443/announce"
,"https://tracker.nanoha.org:443/announce"
,"https://tracker.lelux.fi:443/announce"
,"https://tracker.imgoingto.icu:443/announce"
,"https://tracker.hama3.net:443/announce"
,"https://tracker.foreverpirates.co:443/announce"
,"http://tracker2.wasabii.com.tw:6969/announce"
,"udp://tracker.sktorrent.net:6969/announce"
,"http://www.wareztorrent.com:80/announce"
,"udp://bt.xxx-tracker.com:2710/announce"
,"udp://tracker.eddie4.nl:6969/announce"
,"udp://tracker.grepler.com:6969/announce"
,"udp://tracker.mg64.net:2710/announce"
,"udp://wambo.club:1337/announce"
,"udp://tracker.dutchtracking.com:6969/announce"
,"udp://tc.animereactor.ru:8082/announce"
,"udp://tracker.justseed.it:1337/announce"
,"udp://tracker.leechers-paradise.org:6969/announce"
,"https://open.kickasstracker.com:443/announce"
,"udp://tracker.coppersurfer.tk:6969/announce"
,"http://87.253.152.137/announce"
,"http://91.216.110.47/announce"
,"http://91.217.91.21:3218/announce"
,"http://91.218.230.81:6969/announce"
,"http://93.92.64.5/announce"
,"http://atrack.pow7.com/announce"
,"http://bt.henbt.com:2710/announce"
,"http://bt.pusacg.org:8080/announce"
,"https://tracker.bt-hash.com:443/announce"
,"udp://tracker.leechers-paradise.org:6969"
,"https://182.176.139.129:6969/announce"
,"udp://zephir.monocul.us:6969/announce"
,"https://tracker.dutchtracking.com:80/announce"
,"https://grifon.info:80/announce"
,"udp://tracker.kicks-ass.net:80/announce"
,"udp://p4p.arenabg.com:1337/announce"
,"udp://tracker.aletorrenty.pl:2710/announce"
,"https://tracker.parrotsec.org:443/announce"
,"https://tracker.moxing.party:6969/announce"
,"https://tracker.ipv6tracker.ru:80/announce"
,"https://tracker.fastdownload.xyz:443/announce"
,"https://gwp2-v19.rinet.ru:80/announce"
,"https://tr.kxmp.cf:80/announce"
,"https://explodie.org:6969/announce"]



###Add tracker single

In [ ]:
trackers = handle.trackers()

next_tier = trackers[len(trackers)-1]['tier']

for x in range(len(new_trackers)):
  annouce_entry = {'url': new_trackers[x], 'tier': next_tier}
  handle.add_tracker(annouce_entry) 
  print(annouce_entry)

###Add trackers multi

In [ ]:
for h in downloads:
  trackers = h.trackers()

  next_tier = trackers[len(trackers)-1]['tier']

  for x in range(len(new_trackers)):
    annouce_entry = {'url': new_trackers[x], 'tier': next_tier}
    h.add_tracker(annouce_entry) 
    print(annouce_entry)

###List trackers

In [ ]:
trackers = handle.trackers()
print("Trackers count ",len(trackers))


In [ ]:
trackers = handle.trackers()
print("Trackers count ",len(trackers))

for x in trackers:
  #print(x)
  print(x['url'], x['tier'])

###Start Torrent

In [ ]:
import time
from IPython.display import display
import ipywidgets as widgets


state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

ses.resume()
for x in downloads:
  x.resume()
  
session_state = ""
if ses.is_paused() :
  session_state = "SESSION IS PAUSED"
else:
  session_state = "SESSION IS RUNNING"

print("            " + session_state)

layout = {'width':'auto', 'margin-left':'50px'}
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_finished():
            s = download.status()
            rate = ""
            rateMB = round(s.download_rate / 1000000, 2)
            if rateMB >= 1:
              rate= "{:.2f} MB".format(rateMB)
            else:
              rate = '{:3d} KB'.format (round(s.download_rate / 1000))

            bar.description = " ".join(
                [
            download.name()+"         ",
                    str(),
                    rate,
                    state_str[s.state],
                ]
            )
            bar.value = round(s.progress * 100)
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            download_bars.remove(bar)
            bar.value = 100
            print(download.name(), "complete")
    time.sleep(1)

###Check status

In [ ]:
from IPython.display import display
import ipywidgets as widgets

session_state = ""
if ses.is_paused() :
  session_state = "SESSION IS PAUSED"
else:
  session_state = "SESSION IS RUNNING"

print("            " + session_state)

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = {'width':'auto', 'margin-left':'50px'}
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

for index, download in enumerate(downloads[:]):
    bar = download_bars[index]
    s = download.status()
    state = ""
    if s.paused == True:
      state = "PAUSED"
    else:
      state = "RUNNING"
    bar.description = " ".join(
        [
            download.name(),
            str(round(s.download_rate / 1000000, 2)),
            "MB/s",
            state,
            state_str[s.state]

        ]
    )
    bar.value = round(s.progress * 100)




            SESSION IS PAUSED


FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

###Pause session

In [ ]:
ses.pause()

###Clear downloads

In [ ]:
for download in downloads:
  ses.remove_torrent(download, 0)

downloads.clear()